# Описание

В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.

## Импорт библиотек

In [1]:
from datetime import datetime

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import (
    train_test_split,  # Импортируем функцию для разделения выборок
)

## Загрузка и подготовка данных

In [2]:
# Загрузка данных и отбор нужных классов
data = load_iris()
# Оставляем только два класса (1 и 2), удаляем класс 0 из набора данных
X = data.data[data.target != 0]
Y = data.target[data.target != 0] - 1  # Нормализация классов: 1 -> 0, 2 -> 1

# Разделение на обучающую и тестовую выборки
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)  # 20% на тестирование

## Создание класса логистической регрессии

In [3]:
# Класс логистической регрессии
class My_LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=10000, optimizer='gradient_descent', **kwargs):
        self.learning_rate = learning_rate  # Скорость обучения
        self.epochs = epochs               # Количество эпох обучения
        self.optimizer = optimizer         # Оптимизатор
        self.kwargs = kwargs               # Дополнительные параметры
        self.theta = None                  # Параметры модели

    # Функция активации сигмоида
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    # Прогноз вероятностей принадлежности к классу.
    def predict_proba(self, X):
        return self.sigmoid(np.dot(X, self.theta))

    # Прогноз класса на основе вероятности.
    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int)

    # Функция потерь (логистическая потеря).
    def loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

    # Обучение модели.
    def fit(self, X, y):
        X = np.c_[np.ones(X.shape[0]), X]  # Добавление единичного столбца для сдвига (bias)
        self.theta = np.zeros(X.shape[1])  # Инициализация параметров нулями

        # Выбор оптимизатора
        if self.optimizer == 'gradient_descent':
            self._fit_gradient_descent(X, y)
        elif self.optimizer == 'rmsprop':
            self._fit_rmsprop(X, y)
        elif self.optimizer == 'nadam':
            self._fit_nadam(X, y)
        else:
            raise ValueError("Unknown optimizer")

    # Градиентный спуск.
    def _fit_gradient_descent(self, X, y):
        for _ in range(self.epochs):
            h = self.predict_proba(X)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.learning_rate * gradient

    # Реализация RMSProp.
    def _fit_rmsprop(self, X, y):
        epsilon = self.kwargs.get('epsilon', 1e-8)
        beta = self.kwargs.get('beta', 0.9)
        cache = np.zeros(X.shape[1])

        for _ in range(self.epochs):
            h = self.predict_proba(X)
            gradient = np.dot(X.T, (h - y)) / y.size
            cache = beta * cache + (1 - beta) * gradient**2
            self.theta -= self.learning_rate * gradient / (np.sqrt(cache) + epsilon)

    # Реализация Nadam (Nesterov-accelerated Adam).
    def _fit_nadam(self, X, y):
        beta1 = self.kwargs.get('beta1', 0.9)
        beta2 = self.kwargs.get('beta2', 0.999)
        epsilon = self.kwargs.get('epsilon', 1e-8)
        m = np.zeros(X.shape[1])
        v = np.zeros(X.shape[1])
        t = 0

        for _ in range(self.epochs):
            t += 1
            h = self.predict_proba(X)
            gradient = np.dot(X.T, (h - y)) / y.size
            m = beta1 * m + (1 - beta1) * gradient
            v = beta2 * v + (1 - beta2) * gradient**2
            m_hat = m / (1 - np.power(beta1, t))
            v_hat = v / (1 - np.power(beta2, t))
            self.theta -= self.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

## Тестрирование

In [4]:
# Тестирование различных методов оптимизации
def run_experiment(optimizer, **kwargs):
    model = My_LogisticRegression(optimizer=optimizer, **kwargs)
    start_time = datetime.now()  # Время начала
    model.fit(X_train, Y_train)  # Обучение модели
    end_time = datetime.now()     # Время окончания
    
    # Рассчет потерь и точности на тестовой выборке
    loss = model.loss(model.predict_proba(np.c_[np.ones(X_test.shape[0]), X_test]), Y_test)
    accuracy = (model.predict(np.c_[np.ones(X_test.shape[0]), X_test]) == Y_test).mean()
    return loss, accuracy, end_time - start_time

In [5]:
# Запуск экспериментов
results = []
for optimizer in ['gradient_descent', 'rmsprop', 'nadam']:
    loss, accuracy, time_taken = run_experiment(optimizer, learning_rate=0.01, epochs=10000, beta=0.9, epsilon=1e-8, beta1=0.9, beta2=0.999)
    results.append((optimizer, loss, accuracy, time_taken))

In [6]:

# Представление результатов в виде DataFrame
df_results = pd.DataFrame(results, columns=['Метод', 'Потеря', 'Точность', 'Время работы'])
print(df_results)

              Метод    Потеря  Точность           Время работы
0  gradient_descent  0.312467      0.85 0 days 00:00:00.116999
1           rmsprop  0.769647      0.85 0 days 00:00:00.147511
2             nadam  1.054709      0.85 0 days 00:00:00.214510


## Вывод

На основе предоставленного кода и выполненного задания по логистической регрессии с использованием различных методов оптимизации, можно сделать следующие выводы:

**Цели эксперимента**: Основная цель заключалась в сравнении эффективности различных методов оптимизации (градиентный спуск, RMSProp и Nadam) для алгоритма логистической регрессии на выборке данных о различных видах ирисов.

**Методология**:
Датасет был загружен из библиотеки Sklearn, отобраны два класса: Iris Versicolor и Iris Virginica.

Логистическая регрессия была реализована вручную в виде класса с методами для обучения и предсказаний.

Применялись три различных метода оптимизации, каждый из которых был отдельно протестирован и проанализирован.

**Результаты**:

В таблице представлены значения потерь и точности для каждого из оптимизаторов:

Градиентный спуск: log_loss — 0.312467, accuracy — 0.85

RMSProp: log_loss — 0.769647, accuracy — 0.85

Nadam: log_loss — 1.054709, accuracy — 0.85

Как видно, при использовании градиентного спуска была достигнута наименьшая потеря, что свидетельствует о более быстром и качественном обучении модели. Несмотря на одинаковую точность, разница в значениях потерь указывает на то, что градиентный спуск справляется с этой задачей более эффективно.

**Время работы**: Методы оптимизации также показали различия по времени работы:

Градиентный спуск: 0.116999 сек.

RMSProp: 0.147511 сек.

Nadam: 0.214510 сек.

Градиентный спуск оказался самым быстрым методом, что делает его предпочтительным выбором в данной ситуации.

**Выводы о методах оптимизации**:

Градиентный спуск проявил наилучшие результаты по метрикам качества (потеря и скорость) и является рекомендуемым методом для данной задачи.
RMSProp и Nadam, несмотря на свои преимущества в других задачах или ситуациях, не показали значительного выигрыша в данной конкретной задаче и их использование не оправдывает затрат времени по сравнению с градиентным спуском.